In [1]:
import math
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
from operator import itemgetter
import csv

In [2]:
'''LOAD EMBEDDING'''
def load_embedding(embedding_name):
    File =  open(embedding_name)
    Model = {}
    for line in File:
        split = line.split()
        #print(split[0])
        word = split[0]
        #word = split[2]
        #print(word)
        embedding = np.array([float(val) for val in split[1:]])
        if len(embedding) == 300:
            Model[word] = embedding

    File.close()
    print(embedding_name +" model loaded!")
    print("-----------------------------------------------------------")
    return Model

In [3]:
'''SANITY TEST'''
def sanity_test(embedding_1, embedding_2, output_file):
    synsetList = []
    for synset in embedding_1:
        print(synset)
        if synset.endswith("n"):
            synsetList.append(synset)

    randomSample = np.random.choice(Synset, 10)

    cont = 0
    progress = 0
    for rnd in randomSample:
        maxScore = 0
        maxSynset = "FOO"
        top = []
        for words in embedding_2:
            similarity = cosine_similarity([embedding_1[rnd]], [embedding_2[words]])
            top.append({'word':words, 'sim':similarity})
            if(len(top)>10):
                top = sorted(top, key=itemgetter('sim'), reverse=True)
                top.pop()
        
        progress = progress + 1
        print("...",progress,"%") 
        with open('/home/josu/Desktop/wn_Mapped->GN.txt', 'a') as the_file:
            the_file.write("Most similar synset to " + str(rnd) + "\n")
            for elem in top:
                the_file.write(str(elem.get('word')) + " " + str(elem.get('sim')) + "\n")
            the_file.write("\n")
    percentage = cont/len(randomSample)*100
    print("Percentage of success: ", percentage, "%") 

In [4]:
'''WORD SEARCH'''
def word_synset_comparison(embedding_1, embedding_2, word_to_search, filepath):
    word_to_search = "party"
    '''
    search = []
    errorc = 0
    for wSearch in gnModel:
        try:
            #if re.match(wSearch, '/^party$/'):
            if wSearch.endswith(word_to_search) and len(wSearch)==len(word_to_search):
                search.append(gnModel[wSearch])
        except:
            errorc = errorc +1

    print("Number of errors "+ str(errorc))
    '''
    top = []
    for synset in embedding_1:
        similarity = cosine_similarity([embedding_2[word_to_search]], [embedding_1[synset]])
        top.append({'synset':synset, 'sim':similarity})
        if(len(top)>10):
            top = sorted(top, key=itemgetter('sim'), reverse=True)
            top.pop()
    print(str(top.get('synset')))
    with open(filepath, 'a') as the_file:
        the_file.write("Most similar synset to " + str(word_to_search) + "\n")
        for elem in top:
            the_file.write(str(elem.get('synset')) + " " + str(elem.get('sim')) + "\n")
        the_file.write("\n")
    print("Results in " + str(filepath))

In [5]:
'''WORD SEARCH'''
def word_synset_comparison_csv(embedding_1, embedding_2, wordList, csvpath, dictpath="NULL"):
    for word in wordList:
        top = []
        for synset in embedding_1:
            similarity = cosine_similarity([embedding_2[word]], [embedding_1[synset]])
            top.append({'synset':synset, 'sim':similarity})
            if(len(top)>10):
                top = sorted(top, key=itemgetter('sim'), reverse=True)
                top.pop()
        if dictpath != "NULL":
            top = translator(top, dictpath)

        with open(csvpath, "a") as output:
            head = "Most similar synset to " + str(word).upper() + "\n"
            output.write(head)
            dictWriter = csv.DictWriter(output, fieldnames=['synset', 'sim'])
            for element in top:
                dictWriter.writerow(element)
            output.write("\n")
            
    print("Results in " + str(csvpath))

In [6]:
def txt_translator(inputFile, dictionary, outputFile):
    dictFile = open(dictionary)
    dictArray = []
    for line in dictFile:
        split = line.split()
        dictArray.append({'from':split[0], 'to':split[1]})
    dictFile.close()
    
    targetFile = open(inputFile)
    for line in targetFile:
        split = line.split()
        toWrite = []
        for word in split:
            #if word.endswith("-n"):
            if re.search('-[a-z]$', word):
                error = 1
                for elem in dictArray:
                    if word == elem.get('from'):
                        error = 0
                        toWrite.append(elem.get('to'))
                if error == 1:
                    print("Could not translate " + word)
                    toWrite.append(word)
            else:
                toWrite.append(word)
        with open(outputFile, 'a') as output:
            for res in toWrite:
                output.write(res + " ")
            output.write("\n")
    targetFile.close()
    
    print("Output file in " + outputFile)

In [7]:
def translator(to_translate, dictionary):
    dictFile = open(dictionary)
    dictArray = []
    for line in dictFile:
        split = line.split()
        dictArray.append({'from':split[0], 'to':split[1]})
    dictFile.close()
    
    if len(to_translate) < 1:
        print("Can't translate, length 0")
        return -1
    
    else:
        transList = []
        for element in to_translate:
            error = 1
            for elem in dictArray:
                if element.get('synset') == elem.get('from'):
                    error = 0
                    transList.append({'synset':elem.get('to'), 'sim':element.get('sim')})
            if error == 1:
                transList.append({'synset':element.get('synset'), 'sim':element.get('sim')})
                print(str(element.get('synset')) + "not found, will not be translated")
        
        return transList

In [9]:
pathWN_GN = '/home/josu/Desktop/vecmap-master/wn_synsets_Mapped.txt'
pathGN = '/home/josu/Desktop/vecmap-master/GN_1,5M_Mapped.txt'
pathWN_GN_UKB = '/home/josu/Desktop/vecmap-master/wn_synsets_Mapped_GN+UKB.txt'
pathGN_UKB = '/home/josu/Desktop/vecmap-master/GN_1,5M_Mapped_UKB_Mapped_WN.txt'
pathWN_GN_UKB_PPA = '/home/josu/Desktop/vecmap-master/wn_synsets_Mapped_GN+UKB+PPA.txt'
pathGN_UKB_PPA = '/home/josu/Desktop/vecmap-master/GN_1,5M_Mapped_UKB_PPA_Mapped_WN.txt'
pathWN_Dict = '/home/josu/Downloads/wnet30_dict.synset-word.txt'

In [10]:
wnEmbedding = load_embedding(pathWN_GN)
gnEmbedding = load_embedding(pathGN)

/home/josu/Desktop/vecmap-master/wn_synsets_Mapped.txt model loaded!
-----------------------------------------------------------
/home/josu/Desktop/vecmap-master/GN_1,5M_Mapped.txt model loaded!
-----------------------------------------------------------


In [9]:
wnEmbedding = load_embedding(pathWN_GN_UKB)
gnEmbedding = load_embedding(pathGN_UKB)

/home/josu/Desktop/vecmap-master/wn_synsets_Mapped_GN+UKB.txt model loaded!
-----------------------------------------------------------
/home/josu/Desktop/vecmap-master/GN_1,5M_Mapped_UKB_Mapped_WN.txt model loaded!
-----------------------------------------------------------


In [7]:
wnEmbedding = load_embedding(pathWN_GN_UKB_PPA)
gnEmbedding = load_embedding(pathGN_UKB_PPA)

/home/josu/Desktop/vecmap-master/wn_synsets_Mapped_GN+UKB+PPA.txt model loaded!
-----------------------------------------------------------
/home/josu/Desktop/vecmap-master/GN_1,5M_Mapped_UKB_PPA_Mapped_WN.txt model loaded!
-----------------------------------------------------------


In [7]:
sanity_test(wnEmbedding, gnEmbedding,'/home/josu/Desktop/prueba.txt')

In [7]:
word_synset_comparison(wnEmbedding, gnEmbedding, "party", "/home/josu/Desktop/basura.txt")

Results in /home/josu/Desktop/party_comparison_WN_GN+UKB.txt


In [11]:
word_synset_comparison_csv(wnEmbedding, gnEmbedding, ['party', 'bank', 'man', 'mole', 'book', 'wood'], "/home/josu/Desktop/WN_GN.csv", pathWN_Dict)

Results in /home/josu/Desktop/WN_GN.csv
